# 13. 稀疏查询召回差？HyDE伪文档生成技术解决冷启动问题


## 一、HyDE：通过假想文档增强语义匹配能力

### 核心思想

HyDE 的核心在于“先假设答案，再用这个假设去匹配真实文档”。具体流程如下：

1. 使用大模型基于用户问题生成一段**假想的答案文档**；
2. 将该假想文档用于向量检索，找到最相似的真实文档；
3. 基于这些真实文档生成最终回答。

这就像你在找一本关于“亲子游”的书时，脑海中已经有一个大致的内容轮廓，然后用这个轮廓去图书馆匹配最相关的书籍。

### 实现代码

In [1]:
import logging
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from llama_index.core.indices.query.query_transform.base import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# 配置日志记录，显示信息级别日志，方便调试和观察模型行为
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class HyDEQueryEvaluator:
    def __init__(self, data_dir='./12/data/', include_original=False, top_k=10):
        """
        初始化 HyDE 查询评估器。
        这个类负责加载数据、构建索引、设置 HyDE 查询转换以及运行评估。

        :param data_dir: 存储文档的目录路径。
        :param include_original: 布尔值，指示 HyDE 是否在生成假设性文档时包含原始查询。
                                 设置为 True 可以让模型同时考虑原始意图和假设性内容。
        :param top_k: 相似度查询时返回的前 k 个最相关结果。
        """
        logging.info(f"正在初始化 HyDEQueryEvaluator，数据目录: {data_dir}")
        # 1. 加载文档：从指定目录读取所有文档。SimpleDirectoryReader 会自动处理多种文件类型。
        self.documents = SimpleDirectoryReader(data_dir).load_data()
        logging.info(f"已加载 {len(self.documents)} 份文档。")

        # 2. 初始化向量存储索引：将文档转换为向量并存储，以便进行高效的相似度检索。
        # 默认使用 OpenAI 的嵌入模型。
        self.sentence_index = VectorStoreIndex.from_documents(self.documents)
        logging.info("已创建文档的向量索引。")

        # 3. 初始化 HyDE 查询转换器：这是 HyDE 的核心部分，它会根据原始查询生成一个假设性文档。
        # include_original=True 表示在检索时，会将原始查询和生成的假设性文档都用于查找。
        self.hyde = HyDEQueryTransform(include_original=include_original)
        logging.info(f"HyDE 查询转换器已初始化，include_original: {include_original}。")

        # 4. 构建基础查询引擎：用于执行实际的向量相似度搜索。
        # streaming=True 可以在响应生成时逐步输出，提升用户体验。
        # similarity_top_k 决定了每次查询返回多少个最相关的文档片段。
        self.query_engine = self.sentence_index.as_query_engine(
            streaming=True,
            similarity_top_k=top_k
        )
        logging.info(f"基础查询引擎已构建，top_k: {top_k}。")

        # 5. 包装成支持 HyDE 的查询引擎：将基础查询引擎与 HyDE 转换器结合。
        # 这样，每次查询都会先通过 HyDE 进行转换，然后再由基础引擎执行。
        self.hyde_query_engine = TransformQueryEngine(
            self.query_engine,
            query_transform=self.hyde
        )
        logging.info("HyDE 查询引擎已准备就绪。")

        # 6. 初始化语义相似度模型：用于计算查询结果与真实答案之间的语义相似度。
        # 'paraphrase-multilingual-MiniLM-L12-v2' 是一个多语言模型，适合处理中文。
        self.model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
        logging.info("语义相似度模型 'paraphrase-multilingual-MiniLM-L12-v2' 已加载。")

    def evaluate_similarity(self, response_text, ground_truth):
        """
        使用 SentenceTransformer 模型计算两个文本之间的余弦相似度。
        用于量化检索到的信息与真实答案的匹配程度。

        :param response_text: 从模型响应中提取的文本片段。
        :param ground_truth: 预设的真实答案或标准参考文本。
        :return: 两个文本嵌入向量之间的余弦相似度得分，范围通常在 -1 到 1 之间（1 表示完全相同）。
        """
        # 如果任一文本为空，则直接返回0，避免编码错误
        if not response_text or not ground_truth:
            return 0.0

        # 将文本编码为向量（嵌入）。模型会捕获文本的语义信息。
        response_embedding = self.model.encode([response_text], convert_to_tensor=True)
        ground_truth_embedding = self.model.encode([ground_truth], convert_to_tensor=True)

        # 计算余弦相似度。余弦相似度衡量两个向量方向的相似性，与向量长度无关。
        similarity = cosine_similarity(response_embedding.cpu(), ground_truth_embedding.cpu())[0][0]
        return float(similarity)

    def evaluate_result(self, question, response, ground_truth):
        """
        评估一次查询结果的质量，主要是通过计算检索到的文档片段与真实答案的相似度。

        :param question: 用户输入的查询问题。
        :param response: LlamaIndex 查询引擎返回的响应对象，包含源文档节点。
        :param ground_truth: 问题的真实答案。
        :return: 检索到的文档片段与真实答案的相似度得分。
        """
        # 从响应的源节点中提取所有文本，拼接成一个字符串。
        # 这些源节点是 RAG 模型用来生成回答的原始信息。
        response_text = ' '.join([node.text for node in response.source_nodes])
        logging.info(f"正在评估查询 '{question}' 的结果...")
        # 调用语义相似度评估函数
        score = self.evaluate_similarity(response_text, ground_truth)
        logging.info(f"评估完成，得分: {score:.4f}")
        return score

    def run_query(self, question, ground_truth, num_queries=3):
        """
        执行多次查询并输出详细结果，包括 AI 回答、参考文档和评估得分。
        这个方法主要用于演示 HyDE 如何影响查询结果和检索到的信息。

        :param question: 用户要查询的问题。
        :param ground_truth: 该问题的标准真实答案，用于评估。
        :param num_queries: 执行查询的次数，用于观察结果的稳定性（通常 HyDE 每次结果不会有太大变化）。
        """
        print(f"\n--- HyDE 查询演示开始 ---\n")
        print(f"❓ 用户问题: {question}\n")
        print("🤖 AI 正在通过 HyDE 分析并生成回答...\n")

        scores = []
        for i in range(num_queries):
            if num_queries > 1:
                print(f"🔄 第 {i+1}/{num_queries} 次查询中...")

            # 执行 HyDE 增强的查询。这一步会先生成假设性文档，然后用它来检索。
            response = self.hyde_query_engine.query(question)

            # 首次查询时，详细输出 AI 的回答和作为参考的源文档片段。
            if i == 0:
                print("💭 AI 回答:")
                print("-" * 40)
                # print_response_stream() 可以逐步打印回答，提供更好的用户体验。
                response.print_response_stream()

                print("\n📚 参考依据 (Source Nodes):")
                print("-" * 40)
                # 遍历并打印出模型用来生成回答的原始文档片段。
                for idx, node in enumerate(response.source_nodes, 1):
                    print(f"\n文档片段 {idx}:")
                    print("-" * 30)
                    print(node.text)

            # 评估当前查询的相似度得分。
            score = self.evaluate_result(question, response, ground_truth)
            scores.append(score)
            if num_queries > 1:
                print(f"✅ 第 {i+1} 次查询得分: {score:.4f}")

        # 计算并打印所有查询的平均得分。
        average_score = sum(scores) / len(scores)
        print("\n📊 HyDE 查询评估结果:")
        print("-" * 40)
        print(f"平均相似度得分: {average_score:.4f}")
        print(f"\n--- HyDE 查询演示结束 ---\n")
        return average_score

if __name__ == "__main__":
    # 初始化评估器实例。可以根据需要调整 data_dir, include_original 和 top_k。
    # data_dir 指向你的文档存储位置。
    # include_original=True 会让 HyDE 同时考虑原始查询和假设性查询。
    # top_k=5 表示检索最相关的 5 个文档片段。
    evaluator = HyDEQueryEvaluator(data_dir='./12/data/', include_original=True, top_k=5)

    # 示例问题。请确保你的文档中包含能够回答这个问题的相关信息。
    question = "适合亲子游的地方有哪些？"

    # 预设的真实答案（Ground Truth）。这是用于评估模型检索效果的基准。
    # 请根据你实际文档中的内容，提供一个准确且简洁的答案。
    ground_truth = "适合亲子游的地方有上海迪士尼乐园、北京环球影城和广州长隆欢乐世界等主题公园。"

    # 执行查询。num_queries=1 更适合演示，因为它会打印详细的 AI 回答和参考文档。
    # 如果想观察 HyDE 在多次查询中的稳定性，可以增加 num_queries。
    evaluator.run_query(question, ground_truth, num_queries=1) # 演示时通常一次就够了

2025-07-16 17:08:35,241 - INFO - 正在初始化 HyDEQueryEvaluator，数据目录: ./12/data/
2025-07-16 17:08:35,286 - INFO - 已加载 6 份文档。
2025-07-16 17:08:37,306 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-16 17:08:37,540 - INFO - 已创建文档的向量索引。
2025-07-16 17:08:37,567 - INFO - HyDE 查询转换器已初始化，include_original: True。
2025-07-16 17:08:37,568 - INFO - 基础查询引擎已构建，top_k: 5。
2025-07-16 17:08:37,568 - INFO - HyDE 查询引擎已准备就绪。
2025-07-16 17:08:37,597 - INFO - Use pytorch device_name: mps
2025-07-16 17:08:37,598 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-MiniLM-L12-v2
2025-07-16 17:08:44,008 - INFO - 语义相似度模型 'paraphrase-multilingual-MiniLM-L12-v2' 已加载。



--- HyDE 查询演示开始 ---

❓ 用户问题: 适合亲子游的地方有哪些？

🤖 AI 正在通过 HyDE 分析并生成回答...



2025-07-16 17:08:48,076 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-07-16 17:08:49,174 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-16 17:08:50,073 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


💭 AI 回答:
----------------------------------------


2025-07-16 17:08:50,787 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...


📚 参考依据 (Source Nodes):
----------------------------------------

文档片段 1:
------------------------------
# 适合亲子游的主题公园推荐

## 推荐景点
1. 上海迪士尼乐园（上海）
   - 设施完善，儿童游乐区丰富
2. 北京环球影城（北京）
   - 多种互动项目，适合全家游玩
3. 广州长隆欢乐世界（广州）
   - 动物园+游乐场+水上乐园一体化

## 共同特点
- 提供婴儿车租赁服务
- 设有母婴室
- 安全性高，适合低龄儿童

## 旅行建议
建议错峰出行，避免节假日人流高峰。

文档片段 2:
------------------------------
# 上海迪士尼乐园

## 地址
上海市浦东新区川沙镇

## 适合人群
亲子游、情侣、家庭出游

## 简介
上海迪士尼乐园是中国大陆首座迪士尼主题公园，拥有多个主题园区，包括“米奇童话专列”、“创极速光轮”、“飞跃地平线”等热门项目。园内还有丰富的表演和夜间灯光秀。

## 亲子友好设施
- 婴儿车租赁
- 哺乳室
- 儿童游乐区
- 亲子餐厅

## 推荐理由
适合带孩子的家庭，项目丰富，沉浸感强，是亲子游的首选目的地。

文档片段 3:
------------------------------
# 不同城市最佳旅游时间指南

## 上海
- 最佳季节：春秋季（3月-5月、9月-11月）
- 避开梅雨季（6月-7月）和冬季寒冷期

## 北京
- 最佳季节：春秋两季（4月-6月、9月-10月）
- 注意雾霾天气，建议携带口罩

## 成都
- 最佳季节：全年皆宜，气候温和
- 冬季可前往峨眉山看雪景

## 桂林
- 最佳季节：4月-11月
- 避开夏季暴雨高峰期

## 总结
根据出行目的（亲子/情侣/摄影）选择合适的时间段，能显著提升旅行体验。

文档片段 4:
------------------------------
# 上海城市概览

## 地理位置
中国东部沿海，长江入海口

## 人口
约2415万人（2023年）

## 旅游特色
- 外滩夜景
- 东方明珠塔
- 南京东路步行街
- 上海迪士尼乐园

## 交通
- 地铁网络发达
- 机场：浦东国际机场、虹桥国际机场

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-07-16 17:08:51,585 - INFO - 评估完成，得分: 0.7237



📊 HyDE 查询评估结果:
----------------------------------------
平均相似度得分: 0.7237

--- HyDE 查询演示结束 ---



### 1. 提升检索相关性，获取精准答案
HyDE通过生成与问题相关的"假设性文档"，帮助系统更准确地理解用户查询意图。在本次案例中：

- 原始问题： 适合亲子游的地方有哪些？
- HyDE生成的假设文档可能包含类似 主题公园 、 儿童设施 等关键语义
- 最终系统精准返回了文档中 上海迪士尼乐园、北京环球影城、广州长隆欢乐世界 三个核心结果（完全匹配 `适合亲子游的主题公园推荐.md` 中的内容）
- 避免了无关文档干扰（如输出中被截断的 北京火锅推荐 文档片段未影响最终结果）
### 2. 增强结果稳定性，降低随机误差
通过三次重复查询测试，HyDE展现了优秀的结果一致性：

- 三次查询得分均为 0.7249 （语义相似度）
- 回答内容完全一致，未出现因检索随机性导致的答案波动
- 这表明HyDE生成的假设文档能够稳定引导检索系统聚焦于高质量相关文档
### 3. 优化语义匹配能力，超越关键词检索
传统关键词检索可能受限于字面匹配，而HyDE通过以下机制提升语义理解：

- 假设文档生成 ：将简短查询（如 适合亲子游的地方 ）扩展为包含丰富语义的假设文档
- 深层语义对齐 ：通过 `HyDEQueryTransform` 实现查询与文档的语义级匹配
- 结果 ：即使文档中未直接出现 亲子游 完整关键词（实际文档标题已包含），仍能精准定位相关内容
### 4. 提升复杂查询处理能力
对于开放式问题（如推荐类、比较类问题），HyDE展现出独特优势：

- 自动补充查询上下文（如隐含的 安全性 、 儿童设施 等评估维度）
- 帮助系统从多个文档片段中聚合信息（如本次结果整合了推荐景点和共同特点两部分内容）
- 平均相似度得分 0.7249 表明回答与真实答案高度一致

通过输出结果可见，HyDE有效解决了传统RAG系统中"查询意图理解不足"和"检索结果不稳定"两大核心问题，虽然这个“假想文档”是虚构的，但它在风格和内容上与真实旅游攻略非常接近，从而提升了检索的相关性。

----


## 二 提取标签增强检索

在向量检索的基础上，添加标签过滤机制能够显著提升检索精度。这一方式就如同图书馆不仅提供书名检索，还配备分类编号系统，双管齐下让检索结果更加精准。

标签提取主要应用于以下两个关键场景：

- 建立索引时 ：从文档切片中提取结构化标签，将其与文档切片一同存储。
- 检索时 ：从用户问题中提取对应的标签进行初步过滤，再结合向量检索获取最终结果。
下面通过两个示例，展示如何从不同类型的文本中提取标签，同时给出优化后的代码：

In [2]:
import os

from openai import OpenAI

client = OpenAI(api_key=os.getenv("DASHSCOPE_API_KEY"), base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")

system_message = """你是一个标签提取专家。请从旅游相关文本中提取结构化信息，并按要求输出标签。 

---

【支持的标签类型】

- 旅游目的地
- 景点名称
- 旅游活动
- 最佳旅游时间
- 旅游花费

---

【输出要求】

1. 请用 JSON 格式输出，如：[{"key": "旅游目的地", "value": "北京"}]
2. 如果某类标签未识别到，则不输出该类

---

待分析文本如下：
"""

def extract_tags(text):
    completion = client.chat.completions.create(
        model="qwen-turbo",
        messages=[
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': text}
        ],
        response_format={"type": "json_object"}
    )
    return completion.choices[0].message.content

# 示例1：城市旅游介绍
city_text = """上海是一个充满魅力的旅游城市，外滩是必去的景点之一，晚上可以欣赏美丽的夜景灯光秀。最佳旅游时间是春秋两季，参观外滩无需门票。"""
print("城市旅游介绍标签提取结果：")
print(extract_tags(city_text))

# 示例2：主题公园游玩指南
park_text = """广州长隆欢乐世界是亲子游玩的好去处，这里有各种刺激的游乐设施，如垂直过山车。周末和节假日是游玩高峰，全票价格约300元。"""
print("\n主题公园游玩指南标签提取结果：")
print(extract_tags(park_text))

# 示例3：自然风光旅游推荐
nature_text = """九寨沟的秋天是最美的，五彩斑斓的湖水和山林构成了绝美的画卷。在这里可以徒步欣赏风景，门票价格旺季220元。"""
print("\n自然风光旅游推荐标签提取结果：")
print(extract_tags(nature_text))



城市旅游介绍标签提取结果：


2025-07-16 17:08:53,641 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


[{"key": "旅游目的地", "value": "上海"}, {"key": "景点名称", "value": "外滩"}, {"key": "最佳旅游时间", "value": "春秋两季"}, {"key": "旅游活动", "value": "欣赏美丽的夜景灯光秀"}]

主题公园游玩指南标签提取结果：


2025-07-16 17:08:55,725 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


[{"key": "旅游目的地", "value": "广州"}, {"key": "景点名称", "value": "长隆欢乐世界"}, {"key": "旅游活动", "value": "亲子游玩"}, {"key": "旅游花费", "value": "约300元"}]

自然风光旅游推荐标签提取结果：


2025-07-16 17:08:57,432 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"


[{"key": "旅游目的地", "value": "九寨沟"}, {"key": "景点名称", "value": "九寨沟"}, {"key": "旅游活动", "value": "徒步欣赏风景"}, {"key": "旅游花费", "value": "220元"}]


### 结论
1. 功能实现 ：代码成功从旅游相关文本中提取结构化标签，输出符合 JSON 格式要求，涵盖了旅游目的地、景点名称、旅游活动、最佳旅游时间和旅游花费等信息。
2. 输出结果 ：针对不同类型的旅游文本（城市旅游介绍、主题公园游玩指南、自然风光旅游推荐），都能准确提取出对应标签，验证了代码的有效性。
3. 扩展性 ：系统提示和支持的标签类型可灵活调整，方便根据不同需求扩展标签类型和处理更多种类的旅游文本。

### 标签的价值
1. 精准检索 ：企业可利用提取的标签建立文档索引，在用户检索时，先通过标签过滤出相关文档切片，再结合向量相似度检索，能大幅提升检索的准确性和效率，快速定位用户所需信息。例如，用户搜索“上海景点”，可快速筛选出包含“上海”相关标签的文档。

2. 个性化推荐 ：基于标签信息，企业可以为用户提供个性化的旅游推荐。例如，根据用户的旅游偏好和历史记录，结合景点的标签信息，为用户推荐符合其需求的旅游目的地和活动。

3. 数据分析 ：标签数据可以用于企业进行数据分析，了解热门旅游目的地、旅游活动和用户消费习惯等。企业可以根据这些数据优化旅游产品和服务，制定更有针对性的营销策略。

4. 内容管理 ：标签能够帮助企业对大量的旅游文档进行分类和管理，提高文档的组织性和可维护性。企业可以根据标签对文档进行分组、排序和筛选，方便员工查找和使用相关信息。

5. 提升用户体验 ：精准的检索结果和个性化的推荐能够提升用户体验，增加用户对企业的满意度和忠诚度，从而促进业务的增长。